# Taller Evaluado de Classificación
## Minería de Datos 2020

### 1.- Cancer Cérvico Uterino

Más de 300.000 mujeres mueren anualmente a causa del Cáncer Cérvico Uterino (CaCu). Para detectar el CaCu se utilizan distintos métodos: el más conocido es el Frotis de Papanicolaou (PAP) que consiste en el raspado de células del cérvix, las que son analizadas en laboratorio para detectar anomalías (Citología). A pesar de todas las cualidades que el PAP ostenta, tiene bastantes falsos positivos y aún mas falsos negativos.  Otro exámen es la colposcopía (de Hinselmann y de Schiller) que son exámenes que realiza el mismo Ginecólogo en su consulta, y por lo tanto propensos a errores. La prueba más concluyente corresponde a la biopsia, pero al ser invasiva, dirigida y costosa, es el último método de confirmación del CaCu. El predecir los resultados de la biopsia basados en los examenes previos y los datos clínicos del paciente, ayuda a focalizar estos exámenes adecuadamente.

En este taller utilizaremos un dataset llamado `cacu.csv` que puede encontrar en el aula (junto a este archivo). Este dataset contiene variables que mayormente son autoexplicativas (e.g., edad, si es fumadora, si utiliza métodos anticonceptivos, etc.). Sin embargo, se utilizan siglas médicas que continuación se detallan:

* IUD - Intra Uterine Device (a form of contraceptive)
* STDs - Sexually Transmitted Diseases
* HPV - Human Papilloma Virus
* HIV - Human Immunodeficiency Virus
* AIDS - Acquired Immunodeficiency Syndrome (caused by HIV)
* CIN - Cervical Intraepithelial Neoplasia
* Dx - Medical Abbreviation for Diagnosis

Este estudio consta de 858 pacientes a las que se le realizaron los 4 exámenes (las 4 últimas columnas: Hinselmann, Schiller, Citology, Biopsy), y se les recolectaron los datos demográficos y clínicos. 

#### **1.1.** Carga de Datos

Naturalmente, no toda la información pudo ser recolectada, por lo que cuando se desconoce algún dato se reemplaza por '?'. Utilice pandas para cargar los datos y reemplazar los '?' por el ya conocido indicador NaN. Reporte cuantas entradas no están disponibles por cada columna.

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

cacu_df = pd.read_csv("cacu.csv")
cacu_df = cacu_df.replace("?",np.NaN)
print(cacu_df.isna().sum())
print("\n")
cacu_df.describe().T["count"]


Age                                     0
Number of sexual partners              26
First sexual intercourse                7
Num of pregnancies                     56
Smokes                                 13
Smokes (years)                         13
Smokes (packs/year)                    13
Hormonal Contraceptives               108
Hormonal Contraceptives (years)       108
IUD                                   117
IUD (years)                           117
STDs                                  105
STDs (number)                         105
STDs:condylomatosis                   105
STDs:cervical condylomatosis          105
STDs:vaginal condylomatosis           105
STDs:vulvo-perineal condylomatosis    105
STDs:syphilis                         105
STDs:pelvic inflammatory disease      105
STDs:genital herpes                   105
STDs:molluscum contagiosum            105
STDs:AIDS                             105
STDs:HIV                              105
STDs:Hepatitis B                  

Age                          858.0
STDs: Number of diagnosis    858.0
Dx:Cancer                    858.0
Dx:CIN                       858.0
Dx:HPV                       858.0
Dx                           858.0
Hinselmann                   858.0
Schiller                     858.0
Citology                     858.0
Biopsy                       858.0
Name: count, dtype: float64

**PREGUNTA**: Elimine las columnas que tienen menos de 50% de datos validos y reporte la cantidad de pacientes que tienen al menos un campo en NaN. ¿Qué puede pasar si eliminamos las características que tienen algunos NaNs? ¿Qué puede pasar si eliminamos a las pacientes que tienen algunos NaNs? 

In [32]:
cacu_df = cacu_df.drop(["STDs: Time since first diagnosis","STDs: Time since last diagnosis"],axis=1)

#### **1.2.** Imputación de Datos

Para evitar perder información valiosa, vamos inputar los datos faltantes. Genere un vector los nombres de las características incompletas (columnas con NaN) que son de tipo binarias, y otro vector con las características de tipo numéricas. Reemplace los NaNs de las características binarias por -1 (representando el concepto de desconocido), y reemplace las variables de tipo numéricascon su media.

In [119]:
columns_numeric=['Age', 'Number of sexual partners', 'First sexual intercourse',
       'Num of pregnancies','Smokes (years)', 'Smokes (packs/year)','Hormonal Contraceptives (years)','IUD (years)','STDs (number)']
columns_binary=['Smokes','Hormonal Contraceptives','IUD','STDs','STDs:condylomatosis',
       'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
       'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
       'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
       'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV','STDs:Hepatitis B','STDs:HPV', 'STDs: Number of diagnosis','Dx:Cancer', 'Dx:CIN', 'Dx:HPV','Dx', 'Hinselmann', 'Schiller',
       'Citology', 'Biopsy']

 **PREGUNTA**: ¿Qué efectos puede tener esta estrategia de imputación? Justifique utilizando los estadísticos descriptivos (``describe()``) de pandas.

In [120]:
cacu_df[columns_binary] = cacu_df[columns_binary].fillna(-1)
for col in columns_numeric:
    cacu_df[col] = cacu_df[col].fillna(cacu_df[col].astype(float).mean())

    
cacu_df.isna().sum()



Age                                   0
Number of sexual partners             0
First sexual intercourse              0
Num of pregnancies                    0
Smokes                                0
Smokes (years)                        0
Smokes (packs/year)                   0
Hormonal Contraceptives               0
Hormonal Contraceptives (years)       0
IUD                                   0
IUD (years)                           0
STDs                                  0
STDs (number)                         0
STDs:condylomatosis                   0
STDs:cervical condylomatosis          0
STDs:vaginal condylomatosis           0
STDs:vulvo-perineal condylomatosis    0
STDs:syphilis                         0
STDs:pelvic inflammatory disease      0
STDs:genital herpes                   0
STDs:molluscum contagiosum            0
STDs:AIDS                             0
STDs:HIV                              0
STDs:Hepatitis B                      0
STDs:HPV                              0


**1.3** Preparación de Datos

Defina un subconjunto de testing del 20% de los datos de forma aleatoria, y separe la característica ``Biopsy`` como la variable target.  

In [103]:
# Ponga su código 

**PREGUNTA**: Genere un diagrama de dispersión entre la edad y el número de embarazos de las pacientes. Distinga en diferentes colores aquellos que pertenecen al conjunto de training y testing. ¿Cree que el testing es representativo? (puede correr el split varias veces). ¿A qué sesgos puede llevarnos un mal split?

In [ ]:
# Ponga su código 

### 2.- Evaluación de Classificadores

Considere los siguientes clasificadores
1. Nearest Neighbors Classifier with 3 neighbors
1. Support Vector Machine with Linear Kernel with $C=0.03$
1. Support Vector Machine with Radial Basis Function with C=1.0, $\gamma=1.5$
1. Gaussian Process Classifier with a squared exponential kernel of $\sigma = 1.0$ and amplitude also $1.0$
1. Random Forest with maximum depth of 6 and 10 estimators 
1. Artificial Neural Network with $\alpha = 1$
1. Naive Bayes
1. Quadratic Discriminant Analysis


#### **2.1.-** Evalue el desempeño de los clasificadores

Imprima la curva ROC de cada clasificador y reporte su accuracy, F1-score y AUC utilizando los conjuntos seleccionados en la pregunta anterior. Además, mida el tiempo de entrenamiento y el tiempo de clasificación de cada uno.

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size=0.2, train_size=0.8, random_state=42)

**PREGUNTA**: Fundamente la selección del mejor clasificador basandose tanto en las 3 métricas de rendimiento como en los tiempos de cómputo. ¿Cómo es posible tener valores elevados de AUC pero bajos en F1-score? ¿Podría pasar lo contrario (altos en F1-score, bajos en AUC)?

In [ ]:
# Ponga su código 

####  **2.2.-** GridSearch

Realice un *gridsearch* para algún parámetro del clasificador seleccionado y reporte los resultados. Muestre las diferencias entre el parámetro original y su mejor resultado ocupando los datos de testing.

In [ ]:
# Ponga su código 

**PREGUNTA**: ¿Puede usted argumentar que el mejor resultado que usted encontró con este *gridsearch* no es debido a simple variabilidad en los resultados?

In [ ]:
# Ponga su código 

### 3. Análisis Libre

Escriba el código que usted estime conveniente para respaldar respuesta la pregunta final de este notebook.


In [ ]:
# Ponga su código 

**PREGUNTA FINAL**: ¿Qué conclusión podría usted sacar respecto al objetivo de predecir el resultado de la biopsia? Imagine ahora que usted es parte de la autoridad sanitaria del país. ¿Tomaría usted una decisión de política pública con este análisis? Fundamente su respuesta basandose no solo en la calidad del análisis, sino que en la calidad de los datos.

In [ ]:
# Ponga su código 